# Vowal Wabbit

Vowpal Wabbit is a flexible open-source project designed to tackle complex interactive machine learning tasks. With Microsoft Research and (earlier) Yahoo! Research as major project contributors, Vowpal Wabbit results from intensive community research and contributions since 2007. It provides you with rapid, online and active machine learning solutions for supervised learning and reinforcement learning. 

 Vowpal Wabbit supports Windows, macOS and Ubuntu operating systems. To date, C#, command line and Python packages of Vowpal Wabbit are available for Windows OS, while Java configuration is yet to be released. For macOS and Ubuntu, C# and Java packages will be out soon.

To read about it more, please refer [this](https://analyticsindiamag.com/guide-to-vowpal-wabbit-a-state-of-the-art-library-for-interactive-machine-learning/) article.

Practical implementation

Here’s a demonstration of solving a contextual bandits problem using Vowpal Wabbit 

Install Python package for Vowpal Wabbit

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy sklearn statsmodels tensorflow keras --user -q

In [ ]:
!python -m pip install boost --user -q
!python -m pip install vowpalwabbit --user -q

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

  Import required libraries 

In [ ]:
import pandas as pd
import sklearn as sk
import numpy as np

Create sample training data

In [ ]:
training_data = [{'action': 1, 'cost': 2, 'prob': 0.3, 'f1': 'a', 'f2': 'c', 'f3': ''},
              {'action': 3, 'cost': 1, 'prob': 0.2, 'f1': 'b', 'f2': 'd', 'f3': ''},
              {'action': 4, 'cost': 0, 'prob': 0.6, 'f1': 'a', 'f2': 'b', 'f3': ''},
              {'action': 2, 'cost': 1, 'prob': 0.4, 'f1': 'a', 'f2': 'b', 'f3': 'c'},
              {'action': 3, 'cost': 2, 'prob': 0.7, 'f1': 'a', 'f2': 'd', 'f3': ''}]


#Where ‘prob’ denotes the probability of the actions’ occurrence, and ‘f’ denotes feature.

#    Convert the above training data in the form of list into a Pandas dataframe.
training_df = pd.DataFrame(training_data)

# Add index to data frame
training_df['index'] = range(1, len(training_df) + 1)
training_df = training_df.set_index("index")

In [ ]:
testing_data = [{'f1': 'b', 'f2': 'c', 'f3': ''},
            {'f1': 'a', 'f2': '', 'f3': 'b'},
            {'f1': 'b', 'f2': 'b', 'f3': ''},
            {'f1': 'a', 'f2': '', 'f3': 'b'}]

testing_df = pd.DataFrame(testing_data)

# Add index to data frame
testing_df['index'] = range(1, len(testing_df) + 1)
testing_df = testing_df.set_index("index")

In [ ]:
training_df

In [ ]:
testing_df

Create a contextual bandit with four possible actions (1,2,3 and 4)

In [ ]:
from vowpalwabbit import pyvw

vw = pyvw.vw("--cb 4")


‘pyvw’ is a Python binding for pylibvw class. –cb is the contextual bandit module for optimizing the predictor based on already existing data without further exploration. ‘4’ in “–cb 4” above denotes the number of possible actions.

Call learn() method for each training example to perform an online update.

In [ ]:
for i in training_df.index:
  action = training_df.loc[i, "action"]
  cost = training_df.loc[i, "cost"]
  probability = training_df.loc[i, "prob"]
  feature1 = training_df.loc[i, "f1"]
  feature2 = training_df.loc[i, "f2"]
  feature3 = training_df.loc[i, "f3"]

  # Construct the example in the required vw format.
  learn_ex = str(action) + ":" + str(cost) + ":" + str(probability) + " | " + str(feature1) + " " + str(feature2) + " " + str(feature3)

  # Here we do the actual learning.
  vw.learn(learn_ex)

Perform predictions on the test set. Construct the examples as done in step (8) but exclude labels and pass them to the predict() method.

In [ ]:
print("test sample  action")
for i in testing_df.index:
  feature1 = testing_df.loc[i, "f1"]
  feature2 = testing_df.loc[i, "f2"]
  feature3 = testing_df.loc[i, "f3"]

  test_ex = "| " + str(feature1) + " " + str(feature2) + " " + str(feature3)
  
  choice = vw.predict(test_ex)
  print("    "+str(i)+"\t\t"+str(choice))